In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
USER_AGENT = (
    'Mozilla/5.0 (X11; Linux x86_64)'
    'AppleWebKit/537.36 (KHTML, like Gecko)'
    'Chrome/90.0.4430.212 Safari/537.36'
)
  
HEADERS = {'User-Agent': USER_AGENT}

## requests.get
https://www.marketwatch.com/investing/future/cl00

In [3]:
website="https://www.marketwatch.com"
req_website= f"{website}/investing/future/cl00/download-data?mod=mw_quote_tab"
page_html = requests.get(website, headers=HEADERS)
page_html.content[0:200]

b'\n\n\n<!DOCTYPE html>\n<html class="">\n<head>\n        <title>MarketWatch: Stock Market News - Financial News - MarketWatch</title>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible"'

## robots.txt

In [4]:
print(requests.get(f"{website}/robots.txt", headers=HEADERS).content.decode())

User-agent: *
Disallow: /2/
Disallow: /3com/
Disallow: /admin/
Disallow: /archive/
Disallow: /bgpoxy/
Disallow: /bin/
Disallow: /cDN_contENt/
Disallow: /cgi-bin/
Disallow: /data/
Disallow: /dbcfiles/
Disallow: /dhtml/
Disallow: /dhtmlmenu/
Disallow: /doubleclick/
Disallow: /dynamiclogic/
Disallow: /email/
Disallow: /feeds/
Disallow: /free/
Disallow: /http2_data/
Disallow: /*?tickers=
Disallow: /login/
Disallow: /MarketPerception/
Disallow: /marketperception/
Disallow: /MyPortfolio/
Disallow: /myportfolio/
Disallow: /news/current/
Disallow: /portfolio/
Disallow: /print/
Disallow: /RealMedia/
Disallow: /realmedia/
Disallow: /registration/
Disallow: /search/
Disallow: /search
Disallow: /story/twittershare
Disallow: /tools/alerts/
Disallow: /tpx/
Disallow: /tools/quotes/secfilings.asp
Disallow: /tools/quotes/form10k.asp
Disallow: /tools/quotes/form10q.asp
Disallow: /tools/quotes/form8k.asp
Disallow: /tools/quotes/registrationstate.asp
Disallow: /tools/quotes/sched13d.asp
Disallow: /tools/q

In [6]:
page = requests.get("https://www.marketwatch.com/investing/future/cl00/download-data?startDate=11/13/2020&endDate=05/13/2021",HEADERS)
soup = bs(page.content, 'lxml')
## column_names
table0=soup.find('table','table table--overflow align--center')
all_column_names =[]
# date
column1 = table0.find("thead").find("th").find("div","cell__content fixed--cell")
all_column_names.append(column1.string)   
#column names
columns = table0.find("thead").find_all("th")
column_names = [c.string for c in columns if c.string is not None]
all_column_names.extend(column_names)
all_column_names

['Date', 'Open', 'High', 'Low', 'Close']

In [9]:
## row data
table1_rows = table0.find("tbody").find_all("tr")
table1 = []
all_tables = []
for r in table1_rows:
    td = r.find_all("td")
    for tr in td:
        if tr.string is  None:
            data=tr.find("div").string
            table1.append(data)
        else:
            row = tr.string
            table1.append(row)
table1

['05/21/2021',
 '$61.89',
 '$64.07',
 '$61.56',
 '$63.58',
 '05/20/2021',
 '$63.35',
 '$63.95',
 '$61.67',
 '$61.94',
 '05/19/2021',
 '$65.30',
 '$65.35',
 '$61.95',
 '$63.35',
 '05/18/2021',
 '$66.33',
 '$67.01',
 '$64.11',
 '$65.49',
 '05/17/2021',
 '$65.50',
 '$66.43',
 '$64.83',
 '$66.27',
 '05/14/2021',
 '$63.83',
 '$65.62',
 '$63.33',
 '$65.37',
 '05/13/2021',
 '$65.75',
 '$65.81',
 '$63.09',
 '$63.82',
 '05/12/2021',
 '$65.46',
 '$66.63',
 '$64.97',
 '$66.08',
 '05/11/2021',
 '$64.90',
 '$65.50',
 '$63.68',
 '$65.28',
 '05/10/2021',
 '$65.57',
 '$65.75',
 '$63.95',
 '$64.92',
 '05/07/2021',
 '$64.90',
 '$65.24',
 '$63.90',
 '$64.90',
 '05/06/2021',
 '$65.27',
 '$65.98',
 '$64.50',
 '$64.71',
 '05/05/2021',
 '$66.45',
 '$66.76',
 '$64.92',
 '$65.63',
 '05/04/2021',
 '$64.53',
 '$66.19',
 '$64.29',
 '$65.69',
 '05/03/2021',
 '$63.64',
 '$64.68',
 '$62.91',
 '$64.49',
 '04/30/2021',
 '$64.88',
 '$64.95',
 '$63.08',
 '$63.58',
 '04/29/2021',
 '$63.70',
 '$65.47',
 '$63.65',
 '$65.01

## change method

In the beginning, I scraped the html content with 'requests' and 'BeautifulSoup', but only the first 20 rows could be grabbed. I attempted to send the time interval request and click the 'next page' button with 'selenium' function. It is unsuccessful. I change to another method. I simulated the process of requesting the data, and checked the request under "Network" (Inspect->Network). Then, a "downloaddata" network is found. Therefore, I use the 'Request URL' to download all data.  
Request URL:   https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate=3%2F1%2F2021&endDate=05%2F14%2F2021&frequency=null&downloadPartial=false&csvDownload=false&newDates=true  

However, this page still shows 20 rows at maximum, even click the ‘Next Page’ button. Thus, the time period must be defined in each URL respectively. I first created the list which contains all required time periods first.


## creat lists for fetch all url
Request URL: https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate=3%2F1%2F2021&endDate=05%2F14%2F2021&frequency=null&downloadPartial=false&csvDownload=false&newDates=true

### list for daily data

In [10]:
list=[]
month=['01','02','03','04','05','06','07','08','09','10','11','12']
# for year 2010-2020
# all date form the first 15 days
for y in range(2010,2021):
    y=str(y)
    for m in month:
        d1='01'
        d2='15'
        url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F{y}&endDate={m}%2F{d2}%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
        list.append(url1)
        
# all date form the rest days           
for y in range(2010,2021):
    for m in month:
        if m in ['01','03','05','07','08','10','12']:
            d1='16'
            d2='31'
            url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F{y}&endDate={m}%2F{d2}%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
            list.append(url1)
        elif m in ['04','06','09','11']:
            d1='16'
            d2='30'
            url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F{y}&endDate={m}%2F{d2}%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
            list.append(url1)
        else:
            if (y % 4) == 0 and (y % 100) !=0 or (y % 400) == 0:
                d1='16'
                d2='29'
                url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F{y}&endDate={m}%2F{d2}%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
                list.append(url1)
            else:
                d1='16'
                d2='28'
                url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F{y}&endDate={m}%2F{d2}%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
                list.append(url1)

# for year 2021 
month1=['01','02','03','04','05']
# all date form the first 15 days
for m in month1:
    d1='01'
    d2='15'
    url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F2021&endDate={m}%2F{d2}%2F2021&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
    list.append(url1)

# all date form the rest days  
for m in month1:
    if m in ['01','03']:
        d1='16'
        d2='31'
        url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F2021&endDate={m}%2F{d2}%2F2021&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
        list.append(url1)
    elif m in ['04']:
        d1='16'
        d2='30'
        url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F2021&endDate={m}%2F{d2}%2F2021&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
        list.append(url1)
    elif m in ['02']:
        d1='16'
        d2='28'
        url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate={m}%2F{d1}%2F2021&endDate={m}%2F{d2}%2F2021&frequency=null&downloadPartial=true&csvDownload=false&newDates=false'
        list.append(url1)
    else:
        pass


### list for monthly data

In [11]:
listm=[]
# year 2010-2020
for y in range(2010,2021):
    y=str(y)
    url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate=01%2F01%2F{y}&endDate=12%2F31%2F{y}&frequency=null&downloadPartial=true&csvDownload=false&newDates=true'
    listm.append(url1)

# year 2021 Jan-April
url1=f'https://www.marketwatch.com/investing/future/cl00/downloaddatapartial?partial=true&index=0&countryCode=&iso=&startDate=01%2F01%2F2021&endDate=04%2F30%2F2021&frequency=null&downloadPartial=true&csvDownload=false&newDates=true'
listm.append(url1)

## grab table 
Pandas read_html

### grab daily table

In [12]:
table_d=pd.DataFrame()
for url in list:
    df=pd.read_html(url)[0].iloc[::-1]
    data=pd.DataFrame(df)
    table_d=pd.concat([table_d,data], axis=0)

In [17]:
table_d.to_csv("oil_price_daily_src.csv", index=False)

### grab monthly table

In [14]:
table_m=pd.DataFrame()
for url in listm:
    df=pd.read_html(url)[2].iloc[::-1]
    data=pd.DataFrame(df)
    table_m=pd.concat([table_m,data], axis=0)

In [15]:
table_m.to_csv("oil_price_monthly_src.csv", index=False)